In [ ]:
import pysam
import os
import sys
from sys import getsizeof
import time
import pandas as pd
import polars as pl
from collections import defaultdict

from collections import defaultdict
import pandas as pd
#from matplotlib import pyplot as plt
import numpy as np
import time
from multiprocessing import Pool
import multiprocessing
from tqdm import tqdm


from glob import glob

directory_path = os.path.abspath(os.path.join('../src/'))
if directory_path not in sys.path:
    sys.path.append(directory_path)
    
from read_process import incorporate_replaced_pos_info,incorporate_insertions_and_deletions,\
get_positions_from_md_tag,reverse_complement,get_edit_information,get_edit_information_wrapper,\
has_edits,get_total_coverage_for_contig_at_position,\
print_read_info, update_coverage_array, get_read_information, get_hamming_distance, remove_softclipped_bases,find

from utils import get_contigs_that_need_bams_written, get_contig_lengths_dict, get_intervals, index_bam, write_rows_to_info_file, write_header_to_edit_info, \
write_read_to_bam_file, remove_file_if_exists, make_folder, concat_and_write_bams_wrapper, make_edit_finding_jobs

from core import run_edit_identifier, run_bam_reconfiguration

import os, psutil


In [ ]:
multiprocessing.cpu_count()

# Preload which barcodes to use...

In [ ]:
barcodes_list_path = '/projects/ps-yeolab3/ekofman/Sammi/MouseBrainEF1A_SingleCell_EPR_batch2/cellranger/results/ms_hippo_stamp_EIF4A_batch2/outs/filtered_feature_bc_matrix/barcodes.tsv.gz'

In [ ]:
barcode_whitelist = set(pd.read_csv(barcodes_list_path, names=['barcodes']).barcodes.tolist())

In [ ]:
len(barcode_whitelist)

# ~~~~~~~~~~~~~~~~~~
# Multi-processing enabled
# ~~~~~~~~~~~~~~~~~~

# An example on a full 10x bam

#### in 10X's bam file, xf=25 means that read is uniquely mapped to a genome, and was used for counting UMI. So we should only look at reads with xf=25 from the 10X bam.

In [ ]:
#bampath = '/projects/ps-yeolab5/ekofman/Sammi/MouseBrainEF1A_SingleCell_EPR_batch2/filtered_possorted_ms_hippo_stamp_bam/filtered_keep_xf25_possorted_genome_with_header.bam_MD.bam'
bampath = '/projects/ps-yeolab3/ekofman/sailor2/data/groups_0_1_2_3_4_5_6_7_8_9_10_11_merged.bam'

samfile = pysam.AlignmentFile(bampath, "rb")
samfile_header = str(samfile.header)

output_folder = '/projects/ps-yeolab3/ekofman/sailor2/scripts/full_test-highmem_bccontig-subset'
#output_folder = '/projects/ps-yeolab3/ekofman/sailor2/scripts/full_test-highmem_bccontig'

# Print info?
verbose = False 


# How many subcontigs to split each contig into to leverage multi-processing
num_intervals_per_contig = 16

# Go through every read and identify all edits

In [ ]:


overall_label_to_list_of_contents, results, overall_time, overall_total_reads, total_seconds_for_reads = run_edit_identifier(
    bampath, 
    output_folder, 
    barcode_whitelist,
    contigs=['1', '2'],
    num_intervals_per_contig=num_intervals_per_contig, 
    verbose=verbose)


In [ ]:
print("Total time: {} seconds".format(overall_time))
print("Total time: {} minutes".format(overall_time/60))

Memory: 230 Gigabytes

# More helper functions

# Combine all of the reads (string representation) for each barcode
## Groups the results from each sub-contig segment above, for example the reads from the first half of chr1 and those from the second half.

### which finished?

In [ ]:
# Make a subfolder into which the split bams will be placed
split_bams_folder = '{}/split_bams'.format(output_folder)
make_folder(split_bams_folder)
    

In [ ]:
from glob import glob

contigs_to_generate_bams_for = get_contigs_that_need_bams_written(overall_label_to_list_of_contents, split_bams_folder)

In [ ]:
contigs_to_generate_bams_for

### Generate list of jobs to be multiprocessed

In [ ]:
from collections import OrderedDict
import time



In [ ]:
total_bam_generation_time = run_bam_reconfiguration(split_bams_folder, bampath, overall_label_to_list_of_contents, contigs_to_generate_bams_for)

In [ ]:
print("Total time to concat and write bams: {} minutes".format(round(total_bam_generation_time/60)))

# Second loop to get coverage at sites with edits

In [1]:
from glob import glob
import os
import sys
from collections import defaultdict
import pandas as pd
import polars as pl
from multiprocessing import get_context
import time
from multiprocessing import Pool
import multiprocessing
from tqdm import tqdm
import numpy as np



directory_path = os.path.abspath(os.path.join('../src/'))
if directory_path not in sys.path:
    sys.path.append(directory_path)
    
from utils import get_edit_info_for_barcode_in_contig_wrapper

from core import gather_edit_information_across_subcontigs, run_coverage_calculator

output_folder = '/projects/ps-yeolab3/ekofman/sailor2/scripts/full_test-highmem_bccontig-subset'
#output_folder = '/projects/ps-yeolab3/ekofman/sailor2/scripts/full_test-highmem_bccontig'


### Gather the edit information generated for each subcontig, and group by contig so we only have 1 edit information dataframe to process per contig

In [2]:
edit_info_grouped_per_contig_combined = gather_edit_information_across_subcontigs(output_folder)

Grouping edit information outputs by contig...
	0/32...
	10/32...
	20/32...
	30/32...
Done grouping! Concatenating ...
Done concatenating!


### Get coverage at edit positions for each contig

##### Merge across contigs for each barcode???

### This is going at rate of 1 items per seconds... which would take several hours...

In [3]:

results, total_time = run_coverage_calculator(edit_info_grouped_per_contig_combined, output_folder)

100%|█████████████████████████████████████████████| 8/8 [00:54<00:00,  6.77s/it]

Contig 1_A. Loading /projects/ps-yeolab3/ekofman/sailor2/scripts/full_test-highmem_bccontig-subset/split_bams/1_A/1_A.bam.sorted.bam bamfile...
Contig 1_A. Loaded bamfile...
Contig 1_A. Iterating through barcodes...
0/393 barcodes for 1_A...
300/393 barcodes for 1_A...
Contig 1_C. Loading /projects/ps-yeolab3/ekofman/sailor2/scripts/full_test-highmem_bccontig-subset/split_bams/1_C/1_C.bam.sorted.bam bamfile...
Contig 1_C. Loaded bamfile...
Contig 1_C. Iterating through barcodes...
0/343 barcodes for 1_C...
300/343 barcodes for 1_C...
Contig 1_T. Loading /projects/ps-yeolab3/ekofman/sailor2/scripts/full_test-highmem_bccontig-subset/split_bams/1_T/1_T.bam.sorted.bam bamfile...
Contig 1_T. Loaded bamfile...
Contig 1_T. Iterating through barcodes...
0/380 barcodes for 1_T...
300/380 barcodes for 1_T...
Contig 1_G. Loading /projects/ps-yeolab3/ekofman/sailor2/scripts/full_test-highmem_bccontig-subset/split_bams/1_G/1_G.bam.sorted.bam bamfile...
Contig 1_G. Loaded bamfile...
Contig 1_G. Iter

In [4]:
print(total_time/60)

0.9164288566758235


In [5]:
len(results)

8

In [6]:
all_edit_info = pd.concat(results)

In [7]:
all_edit_info.groupby(['ref', 'alt']).count()


barcode  contig  position  read_id  strand  dist_from_end  \
ref alt                                                              
A   C      39124   39124     39124    39124   39124          39124   
    G      26570   26570     26570    26570   26570          26570   
    N        355     355       355      355     355            355   
    T      13808   13808     13808    13808   13808          13808   
C   A      66684   66684     66684    66684   66684          66684   
    G      10246   10246     10246    10246   10246          10246   
    N        260     260       260      260     260            260   
    T      55426   55426     55426    55426   55426          55426   
G   A      22729   22729     22729    22729   22729          22729   
    C      13978   13978     13978    13978   13978          13978   
    N        261     261       261      261     261            261   
    T      42215   42215     42215    42215   42215          42215   
T   A      76711   76711     76711    76711   76711          76711   
    C      18994   18994     18994    18994   18994          18994   
    G      40610   40610     40610    40610   40610          40610   
    N        335     335       335      335     335            335   

         base_quality  mapping_quality  barcode_position  coverage  
ref alt                                                             
A   C           39124            39124             39124     39124  
    G           26570            26570             26570     26570  
    N             355              355               355       355  
    T           13808            13808             13808     13808  
C   A           66684            66684             66684     66684  
    G           10246            10246             10246     10246  
    N             260              260               260       260  
    T           55426            55426             55426     55426  
G   A           22729            22729             22729     22729  
    C           13978            13978             13978     13978  
    N             261              261               261       261  
    T           42215            42215             42215     42215  
T   A           76711            76711             76711     76711  
    C           18994            18994             18994     18994  
    G           40610            40610             40610     40610  
    N             335              335               335       335

In [8]:
total_time/60

0.9164288566758235

In [9]:
all_edit_info.to_csv('{}/all_edit_info.tsv'.format(output_folder), sep='\t')

# Group by site to get final total edit and coverage counts at each site

In [ ]:
all_edit_info.head()


# Verify C>T ratios

In [ ]:
all_edit_info.groupby(['ref', 'alt']).count()


In [ ]:
all_edit_info.groupby(['ref', 'alt']).count().plot(kind='barh', legend=False)
plt.title("All edits")

base_quality_thresh = 15
all_edit_info[all_edit_info.base_quality > base_quality_thresh].groupby(['ref', 'alt']).count().plot(kind='barh', legend=False)
plt.title("Edits with base quality > {}".format(base_quality_thresh))

all_edit_info_filtered = all_edit_info[all_edit_info.base_quality > base_quality_thresh]


In [ ]:
example_new_ct =  all_edit_info_filtered[(all_edit_info_filtered.ref == 'C') & (all_edit_info_filtered.alt == 'T')].sort_values('position')

In [ ]:
len(example_new_ct)

In [ ]:
example_new_ct

# Cells that do have STAMP expressed versus don't...?

In [ ]:
stamp_expression_path = \
'/projects/ps-yeolab3/ekofman/Sammi/MouseBrainEF1A_SingleCell_EPR_combined/\
4.1_cells_with_middling_stamp/stamp_expression_for_all_used_cells.tsv'

In [ ]:
stamp_expression_df = pd.read_csv(stamp_expression_path, sep='\t', index_col=0)

In [ ]:
stamp_expression_df.Stamp.hist(bins=50)

In [ ]:
all_edit_info_filtered['edit'] = all_edit_info_filtered['ref'] + '>' + all_edit_info_filtered['alt']

In [ ]:
fractions_ct = []
threshs = [0, 0.5, 1, 2, 3, 4, 5, 6, 6.5, 6.6]
for thresh in threshs:
    print(thresh)
    barcodes_at_stamp_thresh = stamp_expression_df[stamp_expression_df.Stamp > thresh].index
    
    all_edit_info_filtered_in_stamp_level = all_edit_info_filtered[
        all_edit_info_filtered.barcode.isin(barcodes_at_stamp_thresh)]
    
    all_edit_info_filtered_in_stamp_level.groupby(['ref', 'alt']).count().plot(kind='barh', legend=False)
    plt.title("Edit Type Distribution for Cells with STAMP expression above {}".format(thresh))
    
    fraction_ct = len(all_edit_info_filtered_in_stamp_level[all_edit_info_filtered_in_stamp_level['edit'] == 'C>T'])/len(all_edit_info_filtered_in_stamp_level)
    fractions_ct.append(fraction_ct)
    
    

In [ ]:
plt.plot(threshs, fractions_ct)
plt.ylabel("Fraction of total edits that are C>T")
plt.xlabel("STAMP expression minimum")
plt.title("Enrichment for C>T edits within cells filtered by STAMP threshold")

In [ ]:
fractions_ct_low = []
threshs = [1, 2, 3, 4, 5, 6, 6.5, 6.6]
for thresh in threshs:
    print(thresh)
    barcodes_at_stamp_thresh = stamp_expression_df[stamp_expression_df.Stamp < thresh].index
    
    all_edit_info_filtered_in_stamp_level = all_edit_info_filtered[
        all_edit_info_filtered.barcode.isin(barcodes_at_stamp_thresh)]
    
    all_edit_info_filtered_in_stamp_level.groupby(['ref', 'alt']).count().plot(kind='barh', legend=False)
    plt.title("Edit Type Distribution for Cells with STAMP expression below {}".format(thresh))
    
    fraction_ct = len(all_edit_info_filtered_in_stamp_level[all_edit_info_filtered_in_stamp_level['edit'] == 'C>T'])/len(all_edit_info_filtered_in_stamp_level)
    fractions_ct_low.append(fraction_ct)

In [ ]:
plt.plot(threshs, fractions_ct_low)
plt.ylabel("Fraction of total edits that are C>T")
plt.xlabel("STAMP expression maximum")
plt.title("Enrichment for C>T edits within cells filtered by STAMP threshold")